In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import json
from underthesea import word_tokenize


import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
device = "cuda" if torch.cuda.is_available() else "cpu"

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity



import re

import transformers
from transformers import AutoTokenizer

from tqdm.notebook import tqdm
from concurrent.futures import ThreadPoolExecutor
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm.notebook import tqdm
from utils.retrieval import BM25Retrieval, TFIDFRetrieval, SentenceRetrieval

In [2]:
class config:
    test_path = '/media/bbsw/Data/Hung-ws/lazy/v2/ise-dsc01-private-test-offcial.json'
    model_name = 'models/model_v7/checkpoint-1000'
    max_length = 256
    batch_size = 64
    num_workers = 32
    check_point = 'models/model_v7/checkpoint-1000'


In [3]:
df = pd.read_json(config.test_path).transpose()

In [4]:
label2id = {'SUPPORTED': 0, 'REFUTED': 1,'NEI': 2}
id2label = {0: 'SUPPORTED', 1:'REFUTED', 2:'NEI'}

In [5]:
model = AutoModelForSequenceClassification.from_pretrained(config.check_point, num_labels=3, label2id=label2id, id2label=id2label)
model.to(device)
tokenizer = AutoTokenizer.from_pretrained(config.model_name)

In [6]:
df['context_tokenizer'] = df['context'].apply(lambda x: word_tokenize(x.lower(), format='text'))
df['claim_tokenizer'] = df['claim'].apply(lambda x: word_tokenize(x.lower(), format='text'))

In [7]:
sentences = SentenceRetrieval()
tfidf = TFIDFRetrieval()

In [8]:
def split_context(context):
    docs = re.split(r'(?<!\d)\.\s*(?![.])', context)
    docs = [doc for doc in docs if doc]
    return docs

In [9]:
def split_context_evidence(context):
    docs = re.split(r'\.\s*(?![.])', context)
    docs = [doc for doc in docs if doc]
    return docs
    # docs = '\n\n'.join(docs)
    # docs = docs.split('\n\n')
    # return [doc.strip() for doc in docs]

In [10]:
def get_topk(query, context, k=5, option=sentences, isProcessbar=False):
    if isProcessbar:
        global pbar
        pbar.update(1)
    docs =split_context(context)

    return '\n'.join(option.get_topk(query, docs))

In [11]:
pbar = tqdm(total=len(df))
# df['top_similarity'] = df.apply(lambda x: get_topk(x['claim_tokenizer'], x['context_tokenizer'], option=sentences, k=5, isProcessbar=True), axis=1)
df['top_tfdif'] = df.apply(lambda x: get_topk(x['claim_tokenizer'], x['context_tokenizer'], option=tfidf, k=5, isProcessbar=True), axis=1)


  0%|          | 0/5396 [00:00<?, ?it/s]

In [12]:
# df = pd.read_csv('data/test.csv')

In [12]:
df.head()

,context,claim,context_tokenizer,claim_tokenizer,top_tfdif
39773,"Sáng 7/6, bà Nguyễn Thị Lệ Thanh, Giám đốc Sở ...",Xuyên suốt thời gian diễn ra Festival biển Nha...,"sáng 7/6 , bà nguyễn_thị_lệ_thanh , giám_đốc s...",xuyên suốt thời_gian diễn ra festival biển nha...,"người đứng đầu sở du_lịch khánh_hòa cho_hay , ..."
37657,"Còn với Công nghệ dệt may, em không biết may h...",Một số người khuyên Nhi Pham theo Công nghệ th...,"còn với công_nghệ dệt_may , em không biết may ...",một_số người khuyên_nhi pham theo công_nghệ th...,một_số người khuyên em theo công_nghệ thực_phẩ...
40265,Nhà nghiên cứu Guangming Tao ở Đại học Khoa họ...,Nhiệt độ da bên dưới lớp cotton cao làm người ...,nhà_nghiên_cứu guangming tao ở đại_học khoa_họ...,nhiệt_độ da bên dưới lớp cotton cao làm người ...,nhiệt_độ da bên dưới vải siêu_mát tăng từ khoả...
47711,"Trong không gian của du thuyền 5 sao, Shynh Ho...",Nhãn hàng sẽ cùng 6 đối tác đề ra những chiến ...,"trong không_gian của du_thuyền 5 sao , shynh h...",nhãn hàng sẽ cùng 6 đối_tác đề ra những chiến_...,nhãn hàng sẽ cùng đối_tác đề ra những chiến_lư...
39930,Pin sinh học là giải pháp mới để cung cấp năng...,Bụi thông minh chỉ cần 5 tế bào vi khuẩn cũng ...,pin sinh_học là giải_pháp mới để cung_cấp năng...,bụi thông_minh chỉ cần 5 tế_bào vi_khuẩn cũng ...,""" chúng_tôi gọi đó là ' bụi thông_minh ' và ch..."


In [13]:
def predict_sample(claim, context):
    example_batch = tokenizer(
        text=claim,
        text_pair=context,
        max_length=config.max_length,
        padding='max_length',
        truncation='only_second',
        return_tensors='pt',
    ).to(device)
    with torch.no_grad():
        output = model(**example_batch)
        logits = output[0]
    pred = id2label[logits.argmax().item()]
    
    return pred

In [14]:
def preprocess_text(text: str) -> str:    
    text = re.sub(r"['\",\.\?:\-!]", "", text)
    text = text.strip()
    text = " ".join(text.split())
    text = text.lower()
    return text

In [15]:
def get_evident(context, claim):
    # docs = re.split(r'(?<!\d)\.\s*(?![.])', context)
    context_list_seq = re.split(r'\.\s*(?![.])', context)
    context_list_seq = [seq for seq in context_list_seq if seq]
    context_list_seq.append(claim)
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform( context_list_seq)
    similarity_scores = cosine_similarity(tfidf_matrix)
    similarity_with_last = cosine_similarity(tfidf_matrix[-1:], tfidf_matrix[:-1])

    evident_idx = np.argsort(similarity_with_last[0])[::-1][0]
    evident = context_list_seq[evident_idx]
    return evident

In [16]:
result = {}
process_bar = tqdm(total=len(df))
for row in df.iterrows():
    idx = str(row[0])
    claim_tokenizer = row[1]['claim_tokenizer']
    context_tokenizer = row[1]['context_tokenizer']
    claim = row[1]['claim']
    context = row[1]['context']
    docs = row[1]['top_tfdif']
    label = predict_sample(claim_tokenizer, docs)
    if label == 'NEI':
        evidence = ""
    else:
        evidence = preprocess_text(tfidf.get_topk(claim, split_context_evidence(context), k=1)[0])
    result[idx] = {
        "verdict": label,
        "evidence": evidence
    }
    process_bar.update(1)

with open('private_result.json', 'w') as f:
    json.dump(result, f, indent=4, ensure_ascii=False)

  0%|          | 0/5396 [00:00<?, ?it/s]

In [17]:
!zip -r results.zip private_result.json

updating: private_result.json (deflated 75%)
